<a href="https://colab.research.google.com/github/cyberust/HumanSynergyAnalysis/blob/main/HumanSynergySemanticAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ======================================================================
#
#  人的シナジー分析システム V2 - セマンティック分析版
#
#  V1からの改善点:
#  - キーワードカウントベースの分析を廃止
#  - sentence-transformers を用いた意味ベクトル（Embedding）による分析を導入
#  - より精緻で信頼性の高いシナジー計算と価値貢献度分析を実現
#
# ======================================================================


# ===============================================================
# STEP 1: 必要ライブラリのインストールと設定
# ===============================================================

# V2で新たに追加：意味ベクトル生成のためのライブラリ
!pip install sentence-transformers --quiet

# V1から継続して使用するライブラリ
!pip install networkx matplotlib seaborn plotly kaleido --quiet
!pip install scikit-learn pandas numpy scipy --quiet

# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

# 基本ライブラリ
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# 数学・統計ライブラリ
from scipy.integrate import odeint
from sklearn.metrics.pairwise import cosine_similarity

# NLP（自然言語処理）ライブラリ
from sentence_transformers import SentenceTransformer

# その他
import re
from datetime import datetime

print("✅ V2: ライブラリのインストールと設定が完了しました")


# ===============================================================
# STEP 2: データ読み込み関数（V1から変更なし）
# ===============================================================

def load_profile_data():
    """Googleドライブからプロフィールデータを読み込む"""
    base_path = "/content/drive/MyDrive/"
    profiles = {}
    names = ["arale_cohen", "yanay_geva", "yasuyuki_sakane"]
    display_names = ["Arale Cohen", "Yanay Geva", "Yasuyuki Sakane"]

    for name, display_name in zip(names, display_names):
        try:
            with open(f"{base_path}{name}_profile.txt", 'r', encoding='utf-8') as f:
                content = f.read()
            profiles[display_name] = content
            print(f"✅ {display_name}のプロフィールを読み込みました")
        except FileNotFoundError:
            print(f"⚠️ {name}_profile.txt が見つかりません。")
            profiles[display_name] = "Profile data not available."
    return profiles

# データ読み込み実行
profiles = load_profile_data()


# ===============================================================
# STEP 3: 【V2の核心】セマンティック・プロフィール分析
# ===============================================================

class SemanticProfileAnalyzer:
    def __init__(self):
        # 事前学習済みの言語モデルをロード
        # 'all-MiniLM-L6-v2' は高速かつ高品質な英語モデル
        print("🤖 言語モデルをロード中...")
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        print("✅ 言語モデルの準備が完了しました")

    def analyze_all_profiles(self, profiles_dict):
        """プロフィールテキストを意味ベクトルに変換し、他の特徴も抽出する"""
        analysis_results = {}
        profile_names = list(profiles_dict.keys())
        profile_texts = list(profiles_dict.values())

        # プロフィール全体を意味ベクトルに一括変換
        print("🧠 全員のプロフィールを意味ベクトルに変換中...")
        embeddings = self.model.encode(profile_texts, show_progress_bar=True)
        print("✅ ベクトル変換が完了しました")

        for i, name in enumerate(profile_names):
            # 経験年数とネットワーク強度はV1のヒューリスティック手法を流用
            # NOTE: これらの抽出方法も、将来的にはより高度化できる
            experience = self._extract_experience_years(profile_texts[i])
            network = self._extract_network_strength(profile_texts[i])

            analysis_results[name] = {
                'embedding': embeddings[i], # 意味ベクトル
                'experience_years': experience,
                'network_strength': network,
                'raw_profile': profile_texts[i]
            }
        return analysis_results

    def _extract_experience_years(self, text):
        """経験年数を抽出"""
        matches = re.findall(r'(\d+)\+?\s*years?', text.lower())
        return max([int(match) for match in matches]) if matches else 10

    def _extract_network_strength(self, text):
        """ネットワーク強度を推定"""
        indicators = ['network', 'ecosystem', 'connections', 'board member', 'partner']
        score = sum(text.lower().count(indicator) for indicator in indicators)
        return min(score * 20, 100)

# プロフィール分析実行
analyzer = SemanticProfileAnalyzer()
analysis_data = analyzer.analyze_all_profiles(profiles)

print("✅ V2: セマンティック・プロフィール分析が完了しました")


# ===============================================================
# STEP 4: 【V2の核心】セマンティック・シナジー計算
# ===============================================================

class SemanticSynergyCalculator:
    def __init__(self, analysis_data):
        self.data = analysis_data
        self.names = list(analysis_data.keys())

    def calculate_skill_complementarity(self):
        """意味ベクトル間のコサイン類似度を用いてスキル補完性を計算"""
        # 全員の意味ベクトルをスタック
        embeddings = np.array([self.data[name]['embedding'] for name in self.names])

        # コサイン類似度を計算 (値が高いほどスキルセットが近い)
        similarity_matrix = cosine_similarity(embeddings)

        # 補完性は (1 - 類似度) で定義
        complementarity_matrix = 1 - similarity_matrix
        return complementarity_matrix

    def calculate_total_synergy_matrix(self):
        """総合シナジーマトリックスを計算"""
        # V2: スキル補完性の計算方法が新しくなった
        comp_matrix = self.calculate_skill_complementarity()

        # V1から流用する計算ロジック
        experiences = [self.data[name]['experience_years'] for name in self.names]
        networks = [self.data[name]['network_strength'] for name in self.names]
        exp_matrix = self._calculate_experience_synergy(experiences)
        net_matrix = self._calculate_network_effects(networks)
        div_matrix = self._calculate_cultural_diversity_bonus()

        # 重み付き統合（調整可能）
        weights = {
            'complementarity': 0.4, # スキル補完性の重要度を上げる
            'experience': 0.2,
            'network': 0.2,
            'diversity': 0.2
        }

        total_synergy = (
            weights['complementarity'] * comp_matrix +
            weights['experience'] * exp_matrix +
            weights['network'] * net_matrix
        ) * div_matrix # 多様性ボーナスは乗算で適用

        # 自分自身とのシナジーは0にする
        np.fill_diagonal(total_synergy, 0)

        return {
            'total_synergy': total_synergy,
            'skill_complementarity': comp_matrix
        }

    # 以下はV1から流用するプライベートメソッド
    def _calculate_experience_synergy(self, experiences):
        synergy_matrix = np.zeros((3, 3))
        for i, j in np.ndindex(synergy_matrix.shape):
            if i != j:
                exp_diff = abs(experiences[i] - experiences[j])
                synergy_matrix[i, j] = np.exp(-0.1 * exp_diff) # 差が小さいほど高い
        return synergy_matrix

    def _calculate_network_effects(self, networks):
        network_matrix = np.zeros((3, 3))
        for i, j in np.ndindex(network_matrix.shape):
            if i != j:
                network_matrix[i, j] = (networks[i] * networks[j]) / 10000.0
        return network_matrix

    def _calculate_cultural_diversity_bonus(self):
        backgrounds = {'Arale Cohen': 'Western', 'Yanay Geva': 'Western', 'Yasuyuki Sakane': 'Eastern'}
        diversity_matrix = np.ones((3, 3))
        for i, name1 in enumerate(self.names):
            for j, name2 in enumerate(self.names):
                if backgrounds[name1] != backgrounds[name2]:
                    diversity_matrix[i, j] = 1.2 # 異なる文化背景に20%のボーナス
        return diversity_matrix

# シナジー計算実行
synergy_calc = SemanticSynergyCalculator(analysis_data)
synergy_results = synergy_calc.calculate_total_synergy_matrix()

print("✅ V2: セマンティック・シナジー計算が完了しました")


# ===============================================================
# STEP 5: 動的システムモデリング（安定版を標準搭載）
# ===============================================================

# 以前の対話で安定化した最終版のモデルをデフォルトとして使用
class DynamicSystemModel:
    def __init__(self, synergy_results, analysis_data):
        self.synergy_matrix = synergy_results['total_synergy']
        self.data = analysis_data
        self.names = list(analysis_data.keys())

    def define_system_parameters(self):
        """システムパラメータを定義（安定性を最優先したロジスティック成長モデル）"""
        alpha = np.array([0.4, 0.35, 0.45]) # 個人の基礎成長率
        beta = self.synergy_matrix * 0.1 # シナジー効果
        K = np.array([12.0, 12.0, 12.0]) # パフォーマンス上限
        return alpha, beta, K

    def system_dynamics(self, state, t, alpha, beta, K):
        """動的システム方程式（結合ロジスティック成長モデル）"""
        x1, x2, x3 = state
        dx1dt = (alpha[0] * x1 + beta[0,1] * x2 + beta[0,2] * x3) * (1 - x1 / K[0])
        dx2dt = (alpha[1] * x2 + beta[1,0] * x1 + beta[1,2] * x3) * (1 - x2 / K[1])
        dx3dt = (alpha[2] * x3 + beta[2,0] * x1 + beta[2,1] * x2) * (1 - x3 / K[2])
        return [dx1dt, dx2dt, dx3dt]

    def simulate_growth(self, initial_state=None, time_horizon=24):
        """成長シミュレーション実行"""
        if initial_state is None:
            initial_state = [1.0, 1.0, 1.0] # ベースラインを1.0に設定
        alpha, beta, K = self.define_system_parameters()
        t = np.linspace(0, time_horizon, time_horizon * 4)
        solution = odeint(self.system_dynamics, initial_state, t, args=(alpha, beta, K))
        return t, solution, (alpha, beta, K)

# 動的システムシミュレーション実行
dynamic_model = DynamicSystemModel(synergy_results, analysis_data)
time_axis, growth_solution, _ = dynamic_model.simulate_growth()

print("✅ V2: 動的システムモデリングが完了しました")


# ===============================================================
# STEP 6: ゲーム理論分析（より意味のある入力で実行）
# ===============================================================
# V2の精緻化されたシナジー値を入力として使用するため、
# ゲーム理論分析の結果も、より現実的なものになることが期待される

class GameTheoryAnalysis:
    def __init__(self, synergy_results, analysis_data):
        self.synergy_matrix = synergy_results['total_synergy']
        self.data = analysis_data
        self.names = list(analysis_data.keys())
        self.n_players = len(self.names)

    def _get_coalition_value(self, coalition_indices):
        """連合の価値を計算する関数"""
        if not coalition_indices:
            return 0

        # 連合内の個人の基礎価値の合計
        base_value = sum((self.data[self.names[i]]['experience_years'] +
                          self.data[self.names[i]]['network_strength'] / 10.0)
                         for i in coalition_indices)

        # 連合内のシナジーの合計
        synergy_value = 0
        if len(coalition_indices) > 1:
            for i in coalition_indices:
                for j in coalition_indices:
                    synergy_value += self.synergy_matrix[i, j]

        # 基礎価値とシナジー価値を統合
        return base_value + synergy_value * 5 # シナジーの価値を強調

    def calculate_shapley_values(self):
        """シャプレー値を計算"""
        from math import factorial
        from itertools import combinations

        shapley_values = np.zeros(self.n_players)
        all_players_set = set(range(self.n_players))

        for i in range(self.n_players):
            for coalition_size in range(self.n_players):
                num_combinations = 0
                total_marginal_contribution = 0

                other_players = list(all_players_set - {i})
                for coalition in combinations(other_players, coalition_size):
                    coalition_with_i = list(coalition) + [i]
                    coalition_without_i = list(coalition)

                    marginal_contribution = self._get_coalition_value(coalition_with_i) - self._get_coalition_value(coalition_without_i)
                    shapley_values[i] += marginal_contribution

        # 正規化
        total_value_all_players = self._get_coalition_value(list(all_players_set))
        current_total_shapley = np.sum(shapley_values)
        if current_total_shapley > 0:
            shapley_values = (shapley_values / current_total_shapley) * total_value_all_players

        return shapley_values

# ゲーム理論分析実行
game_analysis = GameTheoryAnalysis(synergy_results, analysis_data)
shapley_values = game_analysis.calculate_shapley_values()

print("✅ V2: ゲーム理論分析が完了しました")


# ===============================================================
# STEP 7: 予測モデリング（安定した入力で実行）
# ===============================================================

class PredictionModel:
    def __init__(self, growth_solution, time_axis, synergy_results):
        self.growth_data = growth_solution
        self.time_axis = time_axis
        self.synergy_matrix = synergy_results['total_synergy']

    def forecast_business_metrics(self):
        """ビジネス指標予測"""
        performance_data = []
        for t in range(len(self.time_axis)):
            individual_performance = self.growth_data[t]
            # team_synergy の計算式をより安定なものに
            team_synergy = np.sum(self.synergy_matrix * np.outer(individual_performance, individual_performance)) / 2

            # 各指標の計算
            revenue_growth = np.sum(individual_performance) * 10 + team_synergy * 5
            innovation_index = team_synergy * 10
            market_share = min(np.sum(individual_performance) + team_synergy, 40)
            customer_satisfaction = min(75 + team_synergy, 98)

            performance_data.append({
                'time_months': self.time_axis[t],
                'revenue_growth_rate': revenue_growth,
                'innovation_index': innovation_index,
                'market_share_percent': market_share,
                'customer_satisfaction': customer_satisfaction
            })
        return pd.DataFrame(performance_data)

# 予測モデリング実行
prediction_model = PredictionModel(growth_solution, time_axis, synergy_results)
business_forecast = prediction_model.forecast_business_metrics()

print("✅ V2: 予測モデリングが完了しました")


# ===============================================================
# STEP 8: 可視化エンジン（V1からコード流用、描画内容が変化）
# ===============================================================

# V1のVisualizationEngineは、入力データが変わることで
# V2の分析結果を可視化する役割を担う
class VisualizationEngine:
    def __init__(self, analysis_data, synergy_results, growth_solution,
                 time_axis, business_forecast, shapley_values):
        self.analysis_data = analysis_data
        self.synergy_results = synergy_results
        self.growth_solution = growth_solution
        self.time_axis = time_axis
        self.business_forecast = business_forecast
        self.shapley_values = shapley_values
        self.names = list(analysis_data.keys())

    def create_synergy_heatmap(self):
        """総合シナジーヒートマップ"""
        synergy_matrix = self.synergy_results['total_synergy']
        fig = px.imshow(synergy_matrix,
                        labels=dict(x="メンバー", y="メンバー", color="シナジー強度"),
                        x=self.names, y=self.names,
                        text_auto='.3f',
                        color_continuous_scale='Blues')
        fig.update_layout(title_text='V2: 総合シナジーマトリックス（セマンティック版）')
        return fig

    def create_growth_trajectory(self):
        """動的成長軌道"""
        df = pd.DataFrame(self.growth_solution, columns=self.names)
        df['Time'] = self.time_axis
        fig = px.line(df, x='Time', y=self.names,
                      labels={'value': 'パフォーマンス指数', 'Time': '時間（月）'},
                      title='V2: 動的成長軌道シミュレーション（安定版）')
        return fig

    def create_business_metrics_dashboard(self):
        """ビジネス指標ダッシュボード"""
        df = self.business_forecast
        fig = make_subplots(rows=2, cols=2, subplot_titles=('売上成長率', 'イノベーション指数', 'マーケットシェア', '顧客満足度'))
        fig.add_trace(go.Scatter(x=df['time_months'], y=df['revenue_growth_rate'], name='売上成長率'), row=1, col=1)
        fig.add_trace(go.Scatter(x=df['time_months'], y=df['innovation_index'], name='イノベーション指数'), row=1, col=2)
        fig.add_trace(go.Scatter(x=df['time_months'], y=df['market_share_percent'], name='マーケットシェア'), row=2, col=1)
        fig.add_trace(go.Scatter(x=df['time_months'], y=df['customer_satisfaction'], name='顧客満足度'), row=2, col=2)
        fig.update_layout(title_text='V2: ビジネス指標予測ダッシュボード', showlegend=False)
        return fig

    def create_shapley_value_chart(self):
        """シャプレー値による価値貢献度"""
        fig = px.bar(x=self.names, y=self.shapley_values,
                     labels={'x': 'メンバー', 'y': '価値貢献度（シャプレー値）'},
                     text=np.round(self.shapley_values, 2),
                     title='V2: シャプレー値による価値貢献度分析')
        return fig

    def create_network_graph(self):
        """V1のバグを修正したネットワークグラフ"""
        G = nx.Graph()
        for i, name in enumerate(self.names):
            G.add_node(name, size=self.analysis_data[name]['network_strength'])

        synergy_matrix = self.synergy_results['total_synergy']
        for i, j in np.ndindex(synergy_matrix.shape):
            if i < j:
                G.add_edge(self.names[i], self.names[j], weight=synergy_matrix[i, j])

        pos = nx.spring_layout(G, k=1.5, iterations=50)

        edge_traces = []
        for edge in G.edges(data=True):
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            w = edge[2]['weight'] * 15
            edge_traces.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None], mode='lines',
                                        line=dict(width=w, color='rgba(150,150,150,0.5)'),
                                        hoverinfo='text', text=f"Synergy: {w/15:.3f}"))

        node_trace = go.Scatter(x=[pos[n][0] for n in G.nodes()],
                                y=[pos[n][1] for n in G.nodes()],
                                mode='markers+text',
                                text=self.names, textposition="top center",
                                marker=dict(size=[d['size']/5 for n, d in G.nodes(data=True)],
                                            color=['#FF6B6B', '#4ECDC4', '#45B7D1']),
                                hoverinfo='none')

        fig = go.Figure(data=edge_traces + [node_trace],
                        layout=go.Layout(title='V2: チーム相互作用ネットワーク', showlegend=False,
                                         xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                                         yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))
        return fig

    def generate_all_visualizations(self):
        """全ての可視化を生成・表示"""
        print("\n📊 V2: 可視化を生成中...")

        # V2で意味を持つようになったグラフを中心に可視化
        self.create_synergy_heatmap().show()
        self.create_growth_trajectory().show()
        self.create_business_metrics_dashboard().show()
        self.create_shapley_value_chart().show()
        self.create_network_graph().show()

        print("✅ V2: 全ての可視化が完了しました")

# 可視化エンジン実行
viz_engine = VisualizationEngine(analysis_data, synergy_results, growth_solution,
                                 time_axis, business_forecast, shapley_values)

# ===============================================================
# STEP 9: 包括的レポート生成
# ===============================================================
# V2の分析結果に基づいたレポートを生成

class ReportGenerator:
    def __init__(self, analysis_data, synergy_results, shapley_values):
        self.analysis_data = analysis_data
        self.synergy_results = synergy_results
        self.shapley_values = shapley_values
        self.names = list(analysis_data.keys())

    def generate_report(self):
        """分析結果をまとめたサマリーレポートを生成"""
        report_str = "# 人的シナジー分析レポート V2 (セマンティック版)\n\n"

        # 1. 総合シナジー分析
        report_str += "## 1. 総合シナジー分析\n"
        synergy_df = pd.DataFrame(self.synergy_results['total_synergy'],
                                  columns=self.names, index=self.names)
        report_str += "総合シナジーマトリクスは以下の通りです:\n"
        report_str += synergy_df.to_markdown() + "\n\n"

        # 2. スキル補完性分析
        report_str += "## 2. スキル補完性分析（意味ベクトルベース）\n"
        comp_df = pd.DataFrame(self.synergy_results['skill_complementarity'],
                               columns=self.names, index=self.names)
        report_str += "プロフィール全体の意味的な補完性は以下の通りです（1に近いほど補完的）:\n"
        report_str += comp_df.to_markdown() + "\n\n"

        # 3. 価値貢献度分析
        report_str += "## 3. 価値貢献度分析（シャプレー値）\n"
        report_str += "チーム全体で創出される価値に対する各メンバーの貢献度は以下の通りです:\n"
        for name, val in zip(self.names, self.shapley_values):
            report_str += f"- {name}: {val:.3f}\n"
        report_str += "\n"

        # 4. 総括
        report_str += "## 4. 総括\n"
        report_str += "本分析(V2)は、各メンバーのプロフィールの意味的な内容を直接比較することで、より現実に即したシナジーと貢献度を算出しています。\n"
        report_str += "特に、スキル補完性はキーワードの一致ではなく、文章全体のニュアンスを捉えた結果です。\n"

        return report_str

# レポート生成と表示
report_generator = ReportGenerator(analysis_data, synergy_results, shapley_values)
final_report = report_generator.generate_report()
print("\n📋 V2: 分析レポート\n" + "="*50)
print(final_report)

# レポートをファイルに保存
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"/content/drive/MyDrive/synergy_analysis_report_V2_{timestamp}.md"
try:
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(final_report)
    print(f"✅ レポートが保存されました: {filename}")
except Exception as e:
    print(f"⚠️ レポート保存中にエラーが発生しました: {e}")


print("\n\n🎊 全ての分析が完了しました！ 🎊")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 4.2 MB/s eta 0:00:00
Mounted at /content/drive
✅ V2: ライブラリのインストールと設定

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ 言語モデルの準備が完了しました
🧠 全員のプロフィールを意味ベクトルに変換中...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ ベクトル変換が完了しました
✅ V2: セマンティック・プロフィール分析が完了しました
✅ V2: セマンティック・シナジー計算が完了しました
✅ V2: 動的システムモデリングが完了しました
✅ V2: ゲーム理論分析が完了しました
✅ V2: 予測モデリングが完了しました

📊 V2: 可視化を生成中...


✅ V2: 全ての可視化が完了しました

📋 V2: 分析レポート
# 人的シナジー分析レポート V2 (セマンティック版)

## 1. 総合シナジー分析
総合シナジーマトリクスは以下の通りです:
|                 |   Arale Cohen |   Yanay Geva |   Yasuyuki Sakane |
|:----------------|--------------:|-------------:|------------------:|
| Arale Cohen     |      0        |     0.523255 |          0.750359 |
| Yanay Geva      |      0.523255 |     0        |          0.759177 |
| Yasuyuki Sakane |      0.750359 |     0.759177 |          0        |

## 2. スキル補完性分析（意味ベクトルベース）
プロフィール全体の意味的な補完性は以下の通りです（1に近いほど補完的）:
|                 |   Arale Cohen |   Yanay Geva |   Yasuyuki Sakane |
|:----------------|--------------:|-------------:|------------------:|
| Arale Cohen     |  -1.19209e-07 |  0.308137    |       0.563248    |
| Yanay Geva      |   0.308137    | -1.19209e-07 |       0.581619    |
| Yasuyuki Sakane |   0.563248    |  0.581619    |      -2.38419e-07 |

## 3. 価値貢献度分析（シャプレー値）
チーム全体で創出される価値に対する各メンバーの貢献度は以下の通りです:
- Arale Cohen: 41.368
- Yanay Geva: 41.412
- Yasuyuki Sakane: 42.54

In [4]:
print("グラフ1/5: 総合シナジーマトリックスの描画テスト...")
viz_engine.create_synergy_heatmap().show()
print("完了")

グラフ1/5: 総合シナジーマトリックスの描画テスト...


完了


In [5]:
print("グラフ2/5: 動的成長軌道の描画テスト...")
viz_engine.create_growth_trajectory().show()
print("完了")

グラフ2/5: 動的成長軌道の描画テスト...


完了


In [6]:
print("グラフ3/5: ビジネス指標ダッシュボードの描画テスト...")
viz_engine.create_business_metrics_dashboard().show()
print("完了")

グラフ3/5: ビジネス指標ダッシュボードの描画テスト...


完了


In [7]:
print("グラフ4/5: シャプレー値の描画テスト...")
viz_engine.create_shapley_value_chart().show()
print("完了")

グラフ4/5: シャプレー値の描画テスト...


完了


In [8]:
print("グラフ5/5: チーム相互作用ネットワークの描画テスト...")
viz_engine.create_network_graph().show()
print("完了")

グラフ5/5: チーム相互作用ネットワークの描画テスト...


完了
